In [1]:
import requests, os, subprocess
import json
import time

In [ ]:
soft = search_software()
cont = 0
for item in soft[:100]:
    if item != '':
        #print(item)
        search_vulnerabilities(item)
        cont += 1
        if cont > 5:
            time.sleep(30)
            cont = 0
        time.sleep(10)
#
create_item(final)
print(json.dumps(final,indent=2))

Escaneo del software hb_honeypot.pl
Software escaneado, se inicia la concatenación de los cve
Consultando api de vulnerabilidades


In [2]:
#global variables
final = {}

In [3]:
#busca todos los software de un sistema operativo linux
def search_software():
    comand = "ls -ltra /usr/bin | awk '{print $9}'| jq -R . | jq -s ."
    response = subprocess.run(comand,shell=True, stdout=subprocess.PIPE,text=True)
    try:
       data = response.stdout
       data_load = json.loads(data)
       return data_load 
    except json.JSONDecodeError as e:
        print(e)

In [16]:
#main
# esta es la función principal o main, esta concatena todas las demas funciones en un orden específico para que el proceso
# se ejecute de manera exitosa
def search_vulnerabilities(name_software):
    try:
        print(f"Escaneo del software {name_software}")
      #  print(json.dumps(url,indent=2))
        response,valid = search_vulnerabilities_by_keyword(name_software)
        if not valid:
            print('Se eccedió el número de consultas, espera por favor, se retomará en un momento')
            time.sleep(100)
            search_vulnerabilities(name_software) 
        print("Software escaneado, se inicia la concatenación de los cve")
       
        concat= concatResult(response)
            
        mitre = mitres(concat)
       # print('object final:')
        #print(f"mitre {mitre}")
        # se verifica si mitre retorna vacio '{}' de ser así directamente se agrega la key con el nombre del software
        # y en data se especifica que no es vulnerable, eso con el fin de no generar mas llaves y reutilizar la misma data
        if not mitre:
             final[name_software]= {'data': 'No vulnerable'}
        else:
            final[name_software]= {'data ': mitre}
       # print(json.dumps(final,indent=2))
    except requests.exceptions.RequestException as e:
        print(f'Error en la solicitud: {e}')



In [12]:
#apartado de funciones
#concatenar todos los cves dentro de un json creado como vacio, con una llave llamada data que es un array de datos
#este método lee la llave result que arroja la api de vulndb aqui es donde está todos los datos de la vulnerabilidad
#de estos datos solo se usa el id de la vulnerabilidad y el score de la vulnerabilidad
def concatResult(object):
    #cont = 0
        concatCve = {}
        #if 'result' in object: 
        #    for item in object['result'][:3]:
        #         concatCve[item['source']['cve']['id'] ] = item['vulnerability']['risk']
        #         cont += 1
        #    print('concatenado con exito')
        #else:
        #    concatCve = None
        for item in object:
            #print(item)
            concatCve[item['cve']['id']] = {}
        return concatCve
   
        
#esta funcion simplemente retorna el score en en string, para tener una idea más clara
def danger(dat):
    if dat is not None:
        cvss = float(dat)
    else:
        cvss = 0
    if cvss == 0:
        return 'NONE'
    elif 0<cvss<4:
        return 'LOW'
    elif 3.9<cvss<7:
        return 'MEDIUM'
    elif 6.9<cvss<9:
        return 'HIGH'
    elif 8.9<cvss<10.1:
        return 'CRITICAL'
    
             

In [29]:
#apartado de consumo de apis
#aqui se usa una api que es por decirlo así una variante de la api de mitres, la encontré algo más rápida, así que la uso,
#pero cuando se eccede alco asi de las 45 consultas, me arroja error, asi que por esos se le da un delay cuando se hacen 40 consultas
def mitres(object): 
     base_url = "https://cve.circl.lu/api/cve/"
     cont = 0
     max = 0
     jsonMitre = {}
     print("Consultando api de vulnerabilidades")
    # print(f"los datos dle objeto son: {object}")
     if object is not None:
             for key,item in object.items():
                 cont += 1
                 max += 1
                 url = f"{base_url}{key}"
                 if cont > 30:
                     time.sleep(50)
                     cont = 0
                 else:
                     print(f'dato numero {max}')
                # print(url + "\n")
                 if max > 100:
                     print('Se llegó al límite de 100 cve, proceso termiando')
                     break
                
                 try:
                   
                             response = requests.get(url)
                             if response.status_code == 200:
                                 data = response.json()
                                 cvss = 0
                                 if data['cvss'] is not None:
                                     cvss == data['cvss']
                                 else:
                                     cvss == 0
                                 jsonMitre[f'mitre_{key}'] = {'Id':key,
                                                               'Modified':data['Modified'],
                                                               'Published':data['Published'],
                                                               'Description':data['summary'],
                                                               'Score' : {'severity':danger(cvss),'cvss':cvss}}
                                
                                                                
                            # print(json.dumps(jsonMitre,indent=2))
                            # print(json.dumps(data['containerreferences']['cna']['descriptions'],indent=2))
                              
                             else:
                                 print("Error 403, no fue encontrado el servicio")
                    
                         
                 except Exception as e:
                        print(f"Error al consultar la API: {e}")
     else:
        #   print("entra al else")
          jsonMitre = {'status':'No vulnerable'}
       #    print(jsonMitre)
     return jsonMitre
#esta funcion usa la api de nvd que es una api gratuita que limita las peticiones en un rango de 5 por minuto, es por ello que en las
#consultas que se hagana esta funcion se debe especificar un sleep o delay, para que depues de 5 consultas le de un tiempo al servidor
# para que re resfresque nuestras peticiones
def search_vulnerabilities_by_keyword(keyword):
    base_url = "https://services.nvd.nist.gov/rest/json/cves/2.0?keywordSearch="
    query_url = f"{base_url}{keyword}"
    try:
        response = requests.get(query_url)
        response.raise_for_status()
      #  print(query_url)
        
        if response.status_code == 200:
            data = response.json()['vulnerabilities']
            return data,True
        else:
            print("Error al consultar la API del NVD")
            return None,False
    except Exception as e:
            print(f"Error al consultar la API: {e}")

    
    

In [2]:
#Crear items de la base de datos usando api de cosmos, sebe tener en cuenta que la api debe estar en ejecución ya que se 
# ejecuta en un entorno local https://github.com/NicoLa5Tor/COSMOS-LAS
def create_item(object):
    url = 'http://127.0.0.1:5000/create_item?'
    data = {
        "name_Db" : "SoftwareScan",
        "name_item" : "Scan_1",
        "partition" : "100",
        "container" : "Linux",
        "item" : object
    }
    
    try:
        response = requests.post(url,json=data)
        print('Datos guardados')
        print(json.dumps(response.json(),indent=2))
    except Exception as e:
        print(f"Error {e}")
        
    

Datos guardados
{
  "item": {
    "assistant": {
      "data ": {
        "mitre_CVE-1999-1256": {
          "Description": "Oracle Database Assistant 1.0 in Oracle 8.0.3 Enterprise Edition stores the database master password in plaintext in the spoolmain.log file when a new database is created, which allows local users to obtain the password from that file.",
          "Id": "CVE-1999-1256",
          "Modified": "2017-12-19T02:29:00",
          "Published": "1999-03-04T05:00:00",
          "Score": {
            "cvss": 4.6,
            "severity": "MEDIUM"
          }
        },
        "mitre_CVE-2001-1520": {
          "Description": "Xircom REX 6000 allows local users to obtain the 10 digit PIN by starting a serial monitor, connecting to the personal digital assistant (PDA) via Rextools, and capturing the cleartext PIN.",
          "Id": "CVE-2001-1520",
          "Modified": "2008-09-05T20:26:00",
          "Published": "2001-12-31T05:00:00",
          "Score": {
            "cv